In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from uuid import uuid4

# import the .env file
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# configuration
DATA_PATH = r"data"
CHROMA_PATH = r"chroma_db"

In [3]:
from langchain_ollama import OllamaEmbeddings


embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [4]:
vector_store = Chroma(
    collection_name="chatbot",
    embedding_function=embeddings,
    persist_directory=CHROMA_PATH,
)

In [5]:
loader = PyPDFDirectoryLoader(DATA_PATH)

In [6]:
raw_documents = loader.load()

In [7]:
# splitting the document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

# creating the chunks
chunks = text_splitter.split_documents(raw_documents)

In [8]:
uuids = [str(uuid4()) for _ in range(len(chunks))]

# adding chunks to vector store
vector_store.add_documents(documents=chunks, ids=uuids)

['afa90030-ac71-4188-8342-d4b3fef0fe0f',
 'e01e48df-0a01-450e-a34d-e3d858f0cdcb',
 'ccb79a89-fe87-4ba4-be62-1c1edd947222',
 '2f17b1fe-e218-4835-b322-4d8a09a433bd',
 '8f943801-22d6-473e-8db1-d43834d72b2c',
 'b586314f-3eb8-4d51-9761-90f95feefda3',
 'beb840d9-db05-4994-bb3a-9bf49e00f85f',
 '9c1505fa-95a0-4e61-8fba-8b14b48bd0c5',
 '43232b79-8e5a-4de2-a48b-b2acf9fb9181',
 '3dbd6821-da92-4bd1-bba4-594e1adaef7d',
 'a7e8ed74-59fd-46e1-a011-33b108fa40ce',
 '1b8e87e2-e4e0-4f25-bdf6-04b86c7add00',
 '42d6afb7-306f-4ed8-8580-4ef5a85c6470',
 '488b371e-a00c-4ac4-9134-96e0dba19901',
 '79b7870a-0bee-45cf-b664-d17b40f70ed4',
 'c09f0c0a-4a41-43f5-bdf3-e2f6ff9ff5dd',
 'a7237264-7a2b-4c4e-8aaf-fd73f4122062',
 '6dfc2049-527a-4e33-b473-26b0530656a9',
 '84868163-96bf-4786-a10d-c4d97d08dcd4',
 '5ad227e8-678f-4b03-b19c-5bcf922bc692',
 'c146a34e-12a2-40a9-a32a-769222a059c5',
 'e6c16157-a8ce-4852-86ce-f9283a1491e2',
 'fe47623c-841e-4ee9-9614-42c4d6246141',
 'bdd2cccf-8de4-4131-9e45-3863c5d00cd7',
 '4814d8fe-0774-

In [9]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
import gradio as gr

# import the .env file
from dotenv import load_dotenv
load_dotenv()


/home/ezzoubair/Desktop/TTX/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage
# from langchain_ollama import OllamaEmbeddings
from langchain.memory import ConversationBufferMemory
# from langchain_community.document_loaders import TextLoader,PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.chains import LLMChain
# from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
# from langchain_core.vectorstores import InMemoryVectorStore
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
load_dotenv()

GROQ_API_KEY = ""
llm_compression = ChatGroq(temperature=0,api_key=GROQ_API_KEY,model="llama-3.3-70b-versatile")
compressor_llm = LLMChainExtractor.from_llm(llm_compression)

In [11]:
# Set up the vectorstore to be the retriever


num_results = 5
retriever = vector_store.as_retriever(search_kwargs={'k': num_results})



In [ ]:
advanced_retriever = ContextualCompressionRetriever(
    base_compressor=compressor_llm,
    base_retriever=retriever
)

def retrieve_context(query):
    """Récupère le contexte pertinent pour la question"""
    docs = advanced_retriever.get_relevant_documents(query)
    return "\n\n".join([f"- {doc.page_content}" for doc in docs])

# --- Configuration du prompt RAG ---
rag_template = """
Rôle : tu es un expert dans le domaine de la formation et de la formation assistée par l'IA.
Mission : aider l'utilisateur à comprendre et utiliser efficacement une plateforme qui utilise l’IA pour la formation et fournir des réponses sur le domaine de la formation assistée par IA en général.

Historique :
{history}

Connaissances :
{context}

Question :
{question}

Consignes :

    Répondez uniquement avec les informations disponibles

    En cas de hors-sujet : "Je n'ai pas cette information. Voulez-vous une question sur les tendances énergétiques marocaines ?"

    Ton professionnel et clair

Format :
Structure claire avec informations factuelles"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)


llm_response = ChatGroq(temperature=0,model="llama-3.3-70b-versatile",api_key=GROQ_API_KEY)


memory = ConversationBufferMemory(memory_key="history", return_messages=False)

def generate_response(user_message):
    """Génère la réponse de l'assistant"""

    conversation_history = memory.buffer


    context = retrieve_context(user_message)


    prompt = rag_prompt.format(
        history=conversation_history,
        context=context,
        question=user_message
    )


    response = llm_response([HumanMessage(content=prompt)]).content


    memory.save_context({"input": user_message}, {"output": response})

    return response

/tmp/ipykernel_78071/1892361952.py:39: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history", return_messages=False)


In [13]:
import gradio as gr

In [17]:
with gr.Blocks() as demo:
    gr.Markdown("## Djnago ")
    chatbot = gr.Chatbot()
    state = gr.State([])

    with gr.Row():
        txt_input = gr.Textbox(
            show_label=False,
            placeholder="Posez votre question ici...",
            container=False
        )
        send_btn = gr.Button("Envoyer")

    def user_interaction(user_message, history):
        response = generate_response(user_message)
        history.append((user_message, response))
        return "", history

    txt_input.submit(user_interaction, [txt_input, state], [txt_input, chatbot])
    send_btn.click(user_interaction, [txt_input, state], [txt_input, chatbot])

demo.launch()

/tmp/ipykernel_78071/1037708201.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
